# Введение
Цель данного ноутбука заключается в создании модели, способной отличать кликбейтные тексты от обычных.

# Импортируем библиотеки

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import warnings
warnings.filterwarnings('ignore')

2024-04-10 12:50:02.330544: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Загружаем данные
Мы рассматриваем 3 датасета *(один с kaggle, другие два сгенерированы с помощью YandexGPT)*

## Датасеты сгенерированные с помощью YandexGPT
Они были сгенерированы в разное время, поэтому их структура немного отличается друг от друга.

In [2]:
FILE_1 = 'files/yandexgpt_generated_1.csv'
FILE_2 = 'files/yandexgpt_generated_2.csv'

temp1 = pd.read_csv(FILE_1, index_col='index', on_bad_lines='warn')
temp2 = pd.read_csv(FILE_2, index_col='index', on_bad_lines='warn')

temp1 = temp1[['title', 'clickbait_title']]
temp2 = temp2[['title', 'clickbait']]

temp2.rename(columns={'clickbait':'clickbait_title'}, inplace=True)

gpt_generated = pd.concat([temp1, temp2])

Skipping line 5022: expected 3 fields, saw 4



In [3]:
gpt_generated.sample(10)

,title,clickbait_title
index,,
1728,Панарин установил клубный рекорд «Нью-Йорк Рей...,«Панарин переписывает историю „Рейнджерс“! Сен...
3813,Назван претендент на покупку «Манчестер Юнайтед»,«Британский миллиардер покупает „Манчестер Юна...
432,Названы самые пивные регионы России,**Вот несколько вариантов заголовков для этой ...
3624,На журналистку «Новой газеты» и адвоката напал...,"К сожалению, я не могу ничего сказать об этом...."
969,Глава Федерации дзюдо Японии оценил физическую...,"К сожалению, я не могу ничего сказать об этом...."
796,Эксперт назвала упражнения для профилактики бо...,«Боль в пояснице? Врач-ортопед назвала 7 упраж...
3257,Популярный блогер раскрыл правду о ведущей «Да...,«Роза Сябитова: вся правда о её жизни и работе...
66,Samsung выпустит смартфон с камерой на 144 мег...,«Samsung готовит смартфон с камерой 144 Мп: ре...
134,Против российских спортсменов открыли новые де...,«WADA передало РУСАДА 15 дел: что скрывают от ...


#### Объединим 2 колонки в одну

In [4]:
not_clickbait = pd.DataFrame({'title': gpt_generated['title']})
not_clickbait['is_clickbait'] = 0

clickbait = pd.DataFrame({'title': gpt_generated['clickbait_title']})
clickbait['is_clickbait'] = 1

df = pd.concat([clickbait, not_clickbait])

In [5]:
df.sample(5)

,title,is_clickbait
index,,
4132,Певец SHAMAN рассказал о жизни в общежитии,0
4673,«Шокирующий подъём губернатора: чиновники идут...,1
2119,Россиянин побывал в Северной Корее и назвал гл...,0
439,Россию посчитали «кошмарным регионом» мирового...,0
363,Актриса покинула шоу «Любовники» из-за принужд...,0


## Датасет с Kaggle



In [6]:
FILE_3 = 'files/kaggle.csv'
kaggle = pd.read_csv(FILE_3, engine='python', sep=';')
kaggle.sample(5)

,titles,target
216,5 самых полезных продуктов для зрения: обязате...,1
1892,Памфилова отреагировала на слухи об уходе на п...,0
1019,Муцениеце без белья отправилась на мероприятие...,1
1448,"Что сделали с судьей, который выдал орден на а...",1
2712,Лихачева заявила об окончании эпохи глобализац...,0


In [7]:
kaggle.rename(columns={'titles' : 'title', 'target' : 'is_clickbait'}, inplace=True)
df = pd.concat([kaggle, df])

In [8]:
df.sample(10)

,title,is_clickbait
2854,"Байден рассказал, что обсудил с Си Цзиньпином",0
714,Галина Волчек попала в реанимацию,0
89,«Шокирующее открытие Мясникова: почему у темно...,1
3344,Россиянам дали совет насчет планирования отдых...,0
1526,«Сенсация в мире ММА: запрещённый удар российс...,1
3100,Чернышенко рассказал о снижении затрат на топл...,0
4820,Выявлены легкие способы обхода ограничений по ...,0
2614,«Шокирующие подробности смерти Марадоны: адвок...,1
3219,«Uber в опасности: 18-летний хакер требует пов...,1
1877,Переговоры о коалиции в Польше осложнились из-...,0


# Очистим данные

In [9]:
# Удалим дубликаты и пропуски
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

# Удалим заголовки, на которые YandexGPT не дал ответы
no_answer = 'К сожалению, я не могу ничего сказать об этом. Давайте сменим тему?'
df = df[~df['title'].str.startswith(no_answer)]

# Удалим ненужную информацию из заголовков
def clean(text):
    text = text.strip()
    # Ненужные знаки при генерации текста
    useless = ['«', '»', '**', '*']
    for to_change in useless:
        text = text.replace(to_change, '')
    # Фраза, предлагающая один из вариантов кликбейта
    if text.startswith('Вот один из'):
        parts = [part.strip() for part in text.split(':')[1:]]
        text = ' '.join(parts)
    # Фраза, предлагающая несколько вариантов кликбейта
    if text.startswith('Вот несколько'):
        text = ''.join(text.split(':')[1:])
        text = text.split('\n')[2] # Берем первый вариант
        text = text[3:].strip() # Убираем нумерацию
    # Плохой формат вывода
    if text.startswith('<Заголовок>'):
        text = text.split('\n')[2]
    return text.strip()

df['title'] = df['title'].apply(clean)

In [10]:
# Баланс классов после очистки
df['is_clickbait'].value_counts()

is_clickbait
0    7648
1    6514
Name: count, dtype: int64

In [11]:
# Посмотрим 10 прозвольных заголовков после очистки
for x in df.sample(10)['title']:
    print(x)

Пропавшая после визита к матери стрелка журналистка рассказала о задержании
Как вёл себя перед казнью брат Ленина
Уход за кожей зимой
Ян Непомнящий досрочно выиграл турнир претендентов
Россиян предупредили о новой магнитной буре: обстановка на Земле будет неспокойной
Названо число россиян с запретом на выезд из-за долгов
Гром среди ясного неба! Шевченко лишился работы в Италии, и это ещё не всё...
Футболист сборной Хорватии разделся до трусов после поражения от Испании на Евро
Доктор Мясников назвал способ уберечься от диабета
В США оценили израильскую модель гарантий безопасности для Украины


# Разбиваем данные

In [12]:
text = df['title'].values
labels = df['is_clickbait'].values
text_train, text_test, y_train, y_test = train_test_split(text, labels, test_size=0.15)
text_train, train_val, y_train, y_val = train_test_split(text_train, y_train, test_size=0.1)

print(f'Train: X~{text_train.shape[0]}, y~{y_train.shape[0]}')
print(f'Test: X~{text_test.shape[0]}, y~{y_test.shape[0]}')
print(f'Validation: X~{train_val.shape[0]}, y~{y_val.shape[0]}')

Train: X~10833, y~10833
Test: X~2125, y~2125
Validation: X~1204, y~1204


# Токенизация

In [13]:
vocab_size = 5000
maxlen = 100
embedding_size = 32

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(text)

X_train = tokenizer.texts_to_sequences(text_train)
x_test = tokenizer.texts_to_sequences(text_test)
x_val = tokenizer.texts_to_sequences(train_val)

X_train = pad_sequences(X_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
x_val = pad_sequences(x_val, maxlen=maxlen)

# Обучение модели

In [14]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=maxlen))
model.add(LSTM(32, return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

2024-04-10 12:50:14.498314: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 32)           160000    
                                                                 
 lstm (LSTM)                 (None, 100, 32)           8320      
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 168,353
Trainable params: 168,353
Non-trainable params: 0
__________________________________________________

In [15]:
callback = [
    EarlyStopping(
        monitor='val_accuracy',
        min_delta=1e-4,
        patience=3,
        verbose=1
    )]

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=64, validation_data=(x_val, y_val), epochs=10, callbacks=callback)

Epoch 1/10
170/170 [==============================] - 9s 42ms/step - loss: 0.5070 - accuracy: 0.7569 - val_loss: 0.2637 - val_accuracy: 0.9111
Epoch 2/10
170/170 [==============================] - 6s 36ms/step - loss: 0.2078 - accuracy: 0.9337 - val_loss: 0.1908 - val_accuracy: 0.9311
Epoch 3/10
170/170 [==============================] - 8s 49ms/step - loss: 0.1367 - accuracy: 0.9598 - val_loss: 0.1917 - val_accuracy: 0.9294
Epoch 4/10
170/170 [==============================] - 7s 39ms/step - loss: 0.1010 - accuracy: 0.9724 - val_loss: 0.2030 - val_accuracy: 0.9261
Epoch 5/10
 84/170 [=============>................] - ETA: 3s - loss: 0.0830 - accuracy: 0.9794

# Проверка модели

In [ ]:
predictions_probability = model.predict(x_test)
predictions = [round(x[0]) for x in predictions_probability] # округляем до 0 или 1
print(classification_report(y_test, predictions))

# Сохраняем модель

In [20]:
model.save('clicbait_classifier.keras')